In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# List of URLs for different RTC booking pages
URLS = {
    "APSRTC": "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
    "TSRTC": "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
    "RSRTC": "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
    "ASTC": "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
    "CHANTC": "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu",
    "JKSRTC": "https://www.redbus.in/online-booking/jksrtc",
    "KSRTC": "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
    "UPSRTC": "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
    "KTCL": "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
    "SBSTC": "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"
}

# Initialize the driver
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'route')))  # Wait for the routes to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "button")))  # Wait for "View Buses" button

        # Click the "View Buses" button
        view_buses_button = driver.find_element(By.CLASS_NAME, "button")
        driver.execute_script("arguments[0].click();", view_buses_button)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "bus-item")))  # Wait for buses to load

        # Scroll down to load all bus items
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Give time for dynamic loading
        
        # Extract bus details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Compile bus details into a list of dictionaries
        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details
    
    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []

# Main scraping function for each URL
def scrape_all_pages(url, rtc_name):
    driver = initialize_driver()
    load_page(driver, url)
    all_bus_details = []

    for page in range(1, 6):  # Adjust the range based on actual pagination
        try:
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'route')))
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

    driver.quit()
    return all_bus_details

# Scrape data for all RTCs
for rtc_name, url in URLS.items():
    print(f"Scraping data for {rtc_name}...")
    all_bus_details = scrape_all_pages(url, rtc_name)
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(all_bus_details)
    
    # Save the DataFrame to a CSV file named based on the RTC name
    df.to_csv(f'{rtc_name}_bus_details.csv', index=False)
    print(f"Data for {rtc_name} saved to {rtc_name}_bus_details.csv")


In [8]:
import pandas as pd
import pymysql

# List of CSV file paths
csv_files = [
    "APSRTC_bus_details.csv", "ASTC_bus_details.csv", "CHANTC_bus_details.csv",
    "KTCL_bus_details.csv", "KSRTC_bus_details.csv", "RSRTC_bus_details.csv",
    "SBSTC_bus_details.csv", "TSRTC_bus_details.csv", "UPSRTC_bus_details.csv", "JKSRTC_bus_details.csv"
]   

# Read each CSV file into a DataFrame and store it in a list
df_list = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames in the list
combined_df = pd.concat(df_list, ignore_index=True)

# Add id column to the combined DataFrame
combined_df['id'] = range(1, len(combined_df) + 1)

# Save the combined DataFrame to a CSV file
combined_df.to_csv("bus_routes.csv", index=False)

# Load the combined DataFrame from the saved CSV file
df = pd.read_csv("bus_routes.csv")

# Replace 'INR ' in 'Price' column with an empty string and extract digits in 'Seat_Availability' column
df['Price'] = df['Price'].str.replace('INR ', '')
df['Seat_Availability'] = df['Seat_Availability'].str.extract('(\d+)')

# Drop null values
df = df.dropna()

# Connect to MySQL
myconnection = pymysql.connect(host='127.0.0.1', user='root', passwd='Sujan#12345', database="redbus")
cursor = myconnection.cursor()

# Create a MySQL table with appropriate data types
columns_definition = ", ".join(f"{col} {dtype}"
    for col, dtype in zip(df.columns, df.dtypes)
).replace("float64", "FLOAT").replace("object", "TEXT").replace("int64", "INT")

table_name = "bus_routes"
cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_definition})")

# Insert the data into the table row by row
for i in range(len(df)):
    cursor.execute(f"INSERT INTO {table_name} VALUES {tuple(df.iloc[i])}")

# Commit changes and close connection
myconnection.commit()
cursor.close()
myconnection.close()
